In [12]:
import keras
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D,  Flatten, AveragePooling2D,MaxPooling2D, Dropout, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam
import os

In [2]:
#load the mnist dataset
(train_x, train_y) , (test_x, test_y) = cifar10.load_data()
#normalize the data
train_x = train_x.astype('float32') / 255
test_x = test_x.astype('float32') / 255
#print the shapes of the data arrays
print("Train Images: ",train_x.shape)
print("Train Labels: ",train_y.shape)
print("Test Images: ",test_x.shape)
print("Test Labels: ",test_y.shape)
#Encode the labels to vectors
train_y = keras.utils.to_categorical(train_y,10)
test_y = keras.utils.to_categorical(test_y,10)

Train Images:  (50000, 32, 32, 3)
Train Labels:  (50000, 1)
Test Images:  (10000, 32, 32, 3)
Test Labels:  (10000, 1)


In [5]:
#define the model
def MiniModel(input_shape):
  images = Input(input_shape)

  net = BatchNormalization()(images)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same', activation= 'relu')(net)
  net = MaxPooling2D(pool_size= (2,2))(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same', activation= 'relu')(net)
  
  

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same', activation= 'relu')(net)
  net = MaxPooling2D(pool_size= (2,2))(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same', activation= 'relu')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same', activation= 'relu')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same', activation= 'relu')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters= 64, kernel_size= [3,3], strides = [1,1], padding= 'same', activation= 'relu')(net)

  net = Dropout(0.25)(net)
  net = AveragePooling2D(pool_size= (8,8))(net)
  net = Flatten()(net)
  net = Dense(units = 10, activation = 'softmax')(net)

  model = Model(inputs = images, outputs = net)

  return model


input_shape = (32,32,3)
model = MiniModel(input_shape)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 batch_normalization_10 (Ba  (None, 32, 32, 3)         12        
 tchNormalization)                                               
                                                                 
 activation_10 (Activation)  (None, 32, 32, 3)         0         
                                                                 
 conv2d_10 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_11 (Ba  (None, 32, 32, 64)        256       
 tchNormalization)                                               
                                                                 
 activation_11 (Activation)  (None, 32, 32, 64)        0     

In [7]:
#Define the Learning rate schedule function
def lr_schedule(epoch):
  lr = 0.001
  if epoch > 15:
    lr = lr / 100
  elif epoch > 10:
    lr = lr / 10
  elif epoch > 5:
    lr = lr / 5
  print("Learning Rate: ",lr)
  
  return lr

In [8]:
#Pass the scheduler function to the Learning Rate Scheduler class
lr_scheduler = LearningRateScheduler(lr_schedule)

In [10]:
#Directory in which to create models
save_direc = os.path.join(os.getcwd(), 'cifar10savedmodels')

#Name of model files
model_name = 'cifar10model.{epoch:03d}.h5'


#Create Directory if it doesn't exist
if not os.path.isdir(save_direc):
 os.makedirs(save_direc)


#Join the directory with the model file
modelpath = os.path.join(save_direc, model_name)

#Join the directory with the model file
modelpath = os.path.join(save_direc, model_name)

In [11]:
checkpoint = ModelCheckpoint(filepath= modelpath,
                             monitor= 'val_acc',
                             verbose= 1,
                             save_best_only= True,
                             period = 1)

In [13]:
#Specify the training components
model.compile(optimizer=Adam(lr_schedule(0)),loss="categorical_crossentropy",metrics=["accuracy"])

Learning Rate:  0.001


In [14]:
#Fit the model
model.fit(train_x,train_y,batch_size=128,epochs=20,shuffle=True,validation_split=0.1,verbose=1,callbacks=[checkpoint,lr_scheduler])

Learning Rate:  0.001
Epoch 1/20
352/352 [==============================] - 1834s 5s/step - loss: 1.5272 - accuracy: 0.4401 - val_loss: 1.7019 - val_accuracy: 0.4202 - lr: 0.0010
Learning Rate:  0.001
Epoch 2/20
352/352 [==============================] - 1643s 5s/step - loss: 1.0466 - accuracy: 0.6280 - val_loss: 1.2841 - val_accuracy: 0.6004 - lr: 0.0010
Learning Rate:  0.001
Epoch 3/20
352/352 [==============================] - 1537s 4s/step - loss: 0.7898 - accuracy: 0.7251 - val_loss: 0.8208 - val_accuracy: 0.7202 - lr: 0.0010
Learning Rate:  0.001
Epoch 4/20
352/352 [==============================] - 1639s 5s/step - loss: 0.6225 - accuracy: 0.7839 - val_loss: 0.7856 - val_accuracy: 0.7338 - lr: 0.0010
Learning Rate:  0.001
Epoch 5/20
352/352 [==============================] - 3407s 10s/step - loss: 0.5182 - accuracy: 0.8208 - val_loss: 0.7481 - val_accuracy: 0.7552 - lr: 0.0010
Learning Rate:  0.001
Epoch 6/20
352/352 [==============================] - 1887s 5s/step - loss: 0.4380

In [15]:
#Evaluate the accuracy of the test dataset
accuracy = model.evaluate(x=test_x,y=test_y,batch_size=128)
print("Accuracy: ",accuracy[1])

79/79 [==============================] - 83s 1s/step - loss: 0.8048 - accuracy: 0.8293
Accuracy:  0.8292999863624573
